In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import scipy.misc
import pickle

%matplotlib inline

In [2]:
tf.enable_eager_execution()

In [ ]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
def resize_datasets(X):
    X_new = []
    for i in range(X.shape[0]):
        if(i % 100 == 0):
            print(i)
        img = scipy.ndimage.interpolation.zoom(X[i], (8.0, 8.0, 1.0))
        X_new.append(img)
    X_new = np.asarray(X_new, dtype=np.float32)
    return X_new.reshape([X.shape[0], 256, 256, 3])

In [ ]:
print('Resizing Validation...')
X_val = resize_datasets(X_val)

In [ ]:
print('Resizing Test...')
X_test = resize_datasets(X_test)

In [ ]:
print('Resizing Training...')
X_train = resize_datasets(X_train)

In [ ]:
X_train.shape

In [ ]:
pickle_out = open("train_dataset.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("validation_dataset.pickle","rb")
X_val = pickle.load(pickle_in)

In [3]:
import pickle
pickle_in = open("dataset.pickle","rb")
X_train = pickle.load(pickle_in)
y_train = pickle.load(pickle_in)
X_test = pickle.load(pickle_in)
y_test = pickle.load(pickle_in)
X_val = pickle.load(pickle_in)
y_val = pickle.load(pickle_in)

In [4]:
X_test[:30] = X_train[:30]
y_test[:30] = y_train[:30]

In [5]:
training_set_size = 200
test_set_size = 100
valid_set_size = 50

In [6]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

In [7]:
def flatten(x):
    """    
    Input:
    - TensorFlow Tensor of shape (N, D1, ..., DM)
    
    Output:
    - TensorFlow Tensor of shape (N, D1 * ... * DM)
    """
    N = tf.shape(x)[0]
    return tf.reshape(x, (N, -1))

In [ ]:
inp = tf.get_variable(name = 'input', shape=[1,256,256,3], dtype=tf.float32)

In [8]:
#globals used in the assignment
IMG_SHAPE = [256, 256, 3] # height, width, channels
NUM_CLASSES = 10

# training hyperparameters
LEARNING_RATE = 1e-4
MOMENTUM = 0.9
BATCH_SIZE = 16
EPOCHS = 30
DISPLAY_STEP = 1
VALIDATION_STEP = 1000
SAVE_STEP = 1000
CKPT_PATH = './ckpt'
SUMMARY_PATH = './summary'

# net architecture hyperparamaters
LAMBDA = 5e-4 #for weight decay
DROPOUT = 0.5

# test hyper parameters
K_PATCHES = 5
TOP_K = 5

In [9]:
def get_compatibility(atten1_output, g):
#     print("\nCompatibility ->")
#     print("Attention : " + str(atten1_output.numpy().shape)) #(batch, 225, 512)
#     print("G : " + str(g.numpy().shape)) #(batch, 512, 1)
    g_new = tf.reshape(g, (g.shape[0], g.shape[2], g.shape[1])) #(batch, 1, 512)
#     print("After reshaping, G : " + str(g_new.numpy().shape))
    g_new = tf.tile(g_new, [1, atten1_output.shape[1], 1]) #(batch, 225, 512)
    final_vec = (g_new + atten1_output)
#     print("After addition of G and L_i : " + str(final_vec.numpy().shape))
    return final_vec

In [10]:
class AlexNet(tfe.Network):
    def __init__(self, training):
        super(AlexNet, self).__init__()
        self.training = training
        
        # convolutional layers
        conv_init = tf.contrib.layers.xavier_initializer_conv2d()
        
        self.conv1 = self.track_layer(
            tf.layers.Conv2D(
                96,
                11,
                4,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool1 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv2 = self.track_layer(
            tf.layers.Conv2D(
                256,
                5,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool2 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv3 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv4 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv5 = self.track_layer(
            tf.layers.Conv2D(
                256,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool5 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        # fully connected layers

        fc_init = tf.contrib.layers.xavier_initializer()

        self.fc1 = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        self.drop1 = self.track_layer(tf.layers.Dropout(DROPOUT))
        
        #perceptron
        self.perceptron1 = self.track_layer(
            tf.layers.Dense(
                1, kernel_initializer=fc_init))
        
        self.perceptron2 = self.track_layer(
            tf.layers.Dense(
                1, kernel_initializer=fc_init))
        
        #attention estimators
        self.atten1_fc = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        
        self.atten2_fc = self.track_layer(
            tf.layers.Dense(
                512, kernel_initializer=fc_init))
        
        self.out = self.track_layer(
            tf.layers.Dense(NUM_CLASSES, kernel_initializer=fc_init))

    def call(self, x):
        """ Function that executes the model """
        output = self.conv1(x)
        #print("After conv1 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        #print("After normalisation1 : " + str(output.numpy().shape))
        output = self.pool1(output)
        #print("After pool1 : " + str(output.numpy().shape))

        output = self.conv2(output)
        #print("After conv2 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        #print("After normalisation2 : " + str(output.numpy().shape))
        output = self.pool2(output)
        #print("After pool2 : " + str(output.numpy().shape))

        output = self.conv3(output)
        #print("After conv3 : " + str(output.numpy().shape))

        output = self.conv4(output)
        #print("After conv4 : " + str(output.numpy().shape))
        conv4_output = output

        output = self.conv5(output)
        #print("After conv5 : " + str(output.numpy().shape))
        conv5_output = output
        
        output = self.pool5(output)
        #print("After pool5 : " + str(output.numpy().shape))

        output = tf.layers.flatten(output)
        #print("After flattening : " + str(output.numpy().shape))

        output = self.fc1(output)
        #print("After fc1 : " + str(output.numpy().shape))
        if self.training:
            output = self.drop1(output)
            #print("After drop1 : " + str(output.numpy().shape))
        
        #logic for attention estimators
        g = tf.reshape(output, (output.shape[0], output.shape[1], 1))
#         print("G dimensions : " + str(g.numpy().shape))
        
#         print("\nATTENTION 1 :-")
        atten1_flattened_output = tf.reshape(conv4_output, (conv4_output.shape[0], conv4_output.shape[1] * conv4_output.shape[2], conv4_output.shape[3]))
#         print("After Flattening : " + str(atten1_flattened_output.numpy().shape))
        atten1_output = self.atten1_fc(atten1_flattened_output)
#         print("After FC : " + str(atten1_output.numpy().shape))
        atten1_output = self.perceptron1(get_compatibility(atten1_output, g))
#         print("After passing through perceptron : " + str(atten1_output.numpy().shape))
        atten1_output = tf.reshape(atten1_output, (atten1_output.shape[0], atten1_output.shape[2], atten1_output.shape[1]))
        attention1_values = atten1_output
        atten1_output = tf.nn.softmax(atten1_output)
#         print("After Reshaping and applying Softmax : " + str(atten1_output.numpy().shape))
        atten1_output = tf.matmul(atten1_output, atten1_flattened_output)
#         print("G_a1 Dimensions : " + str(atten1_output.numpy().shape))
        
        #print("\nATTENTION 2 :-")
        atten2_flattened_output = tf.reshape(conv5_output, (conv5_output.shape[0], conv5_output.shape[1] * conv5_output.shape[2], conv5_output.shape[3]))
        #print("After Flattening : " + str(atten2_flattened_output.numpy().shape))
        atten2_output = self.atten2_fc(atten2_flattened_output)
        #print("After FC : " + str(atten2_output.numpy().shape))
        atten2_output = self.perceptron2(get_compatibility(atten2_output, g))
        #print("After Dot Product with G : " + str(atten2_output.numpy().shape))
        atten2_output = tf.reshape(atten2_output, (atten2_output.shape[0], atten2_output.shape[2], atten2_output.shape[1]))
        attention2_values = atten2_output
        atten2_output = tf.nn.softmax(atten2_output)
        #print("After Reshaping and applying Softmax : " + str(atten2_output.numpy().shape))
        atten2_output = tf.matmul(atten2_output, atten2_flattened_output)
        #print("G_a1 Dimensions : " + str(atten2_output.numpy().shape))
        
        #print("\nCOMBINING BY OPTION 1 :-")
        output = tf.concat([atten1_output, atten2_output], 2)
        output = tf.reshape(output, (output.shape[0], output.shape[2]))
        #print("Option 1 Dimensions : " + str(output.numpy().shape))
        output = self.out(output)

        return output,attention1_values,attention2_values

In [11]:
DISPLAY_STEP = 1
VALIDATION_STEP = 5
SAVE_STEP = 10

In [12]:
#used for printing the log with the timestamp
logging = tf.logging
logging.set_verbosity(logging.INFO)
def log_msg(msg):
       logging.info(f'{time.ctime()}: {msg}') 

In [13]:
def loss(net, mode, x, y):
    """
    Computes the loss for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the loss between the predictions on the images and the groundtruths
    """
    pred,_,_ = net(x)
    y = tf.one_hot(y, NUM_CLASSES)

    loss_value = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=pred)
    weight_decay = tf.reduce_sum(LAMBDA * tf.stack([tf.nn.l2_loss(v) for v in net.variables]))

    total_loss = loss_value + weight_decay

    tf.contrib.summary.scalar(mode + '/loss', total_loss)

    return total_loss

In [14]:
def accuracy(net, mode, x, y):
    """
    Computes the accuracy for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the accuracy of the predictions on the images and the groundtruths
    """
    pred = tf.nn.softmax(net(x)[0])
    y = tf.one_hot(y, NUM_CLASSES)

    accuracy_value = tf.reduce_sum(
                tf.cast(
                    tf.equal(
                        tf.argmax(pred, axis=1, output_type=tf.int64),
                        tf.argmax(y, axis=1, output_type=tf.int64)
                    ),
                    dtype=tf.float32
                ) 
            ) / float(pred.shape[0].value)

    tf.contrib.summary.scalar(mode +'/accuracy', accuracy_value)

    return accuracy_value

In [282]:
train_dset = Dataset(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=valid_set_size, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=test_set_size)

In [17]:
BATCH_SIZE = 8
train_dset = Dataset(X_train[:8], y_train[:8], batch_size=8, shuffle=True)

In [18]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
#     print(y)
    if t > 5: break

0 (8, 256, 256, 3) (8,)


In [53]:
img = X_val[1,:]
img = tf.to_float(img.reshape([1,256,256,3]))

In [ ]:
trainingset = Dataset(X_val[:8], y_val[:8], batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
BATCH_SIZE = 128

In [21]:
import time
def train(net):
    """
    Training procedure
    """
    start_time = time.time()
    step_time = 0.0
    
    writer = tf.contrib.summary.create_summary_file_writer(SUMMARY_PATH)
#     optimizer = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE, momentum=MOMENTUM)
    optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
    global_step = tf.train.get_or_create_global_step()
    epoch = tfe.Variable(0, name='epoch', dtype=tf.float32, trainable=False)
    all_variables = (net.variables + optimizer.variables() + [global_step] + [epoch])
    
    with writer.as_default():
        with tf.contrib.summary.record_summaries_every_n_global_steps(DISPLAY_STEP):
            for e in range(int(epoch.numpy()), EPOCHS):
                tf.assign(epoch, e)
                for (batch_i, (images, labels)) in enumerate(train_dset):
                    global_step = tf.train.get_global_step()
                    step = global_step.numpy() + 1
                    images = tf.to_float(images)
                    step_start_time = int(round(time.time() * 1000))
                    
                    optimizer.minimize(lambda: loss(net, 'train', images, labels), global_step=global_step)

                    step_end_time = int(round(time.time() * 1000))
                    step_time += step_end_time - step_start_time

                    if (step % DISPLAY_STEP) == 0:
                        l = loss(net, 'train', images, labels)
                        a = accuracy(net, 'train', images, labels).numpy()
                        log_msg('Epoch: {:03d} Step/Batch: {:09d} Step mean time: {:04d}ms \nLoss: {:.7f} Training accuracy: {:.4f}'.format(e, step, int(step_time / step), l, a))

#                     if (step % VALIDATION_STEP) == 0:
#                         for (_, (val_images, val_labels)) in enumerate(val_dset):
# #                             val_images, val_labels = tfe.Iterator(val_dset).next()
#                             l = loss(net, 'val', val_images, val_labels)
#                             a = accuracy(net, 'val', val_images, val_labels).numpy()
#                             int_time = time.time() - start_time
#                             log_msg('Elapsed time: {:04d}ms --- Loss: {:.7f} Validation accuracy: {:.4f}'.format(int(int_time), l, a))

#                     if (step % SAVE_STEP) == 0:
#                         tfe.Saver(net.variables).save(os.path.join(CKPT_PATH, 'net.ckpt'), global_step=global_step)
#                         log_msg('Variables saved')

In [285]:
alexnet = AlexNet(training=True)


Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but note that `track_layer` is no longer necessary or supported. Instead, `Layer` instances are tracked on attribute assignment (see the section of `tf.keras.Model`'s documentation on subclassing). Since the output of `track_layer` is often assigned to an attribute anyway, most code can be ported by simply removing the `track_layer` calls.

`tf.keras.Model` works with all TensorFlow `Layer` instances, including those from `tf.layers`, but switching to the `tf.keras.layers` versions along with the migration to `tf.keras.Model` is recommended, since it will preserve variable names. Feel free to import it with an alias to avoid excess typing :).


In [ ]:
if not os.path.exists(CKPT_PATH):
    os.makedirs(CKPT_PATH)
train(alexnet)

INFO:tensorflow:Sat Nov 17 20:52:51 2018: Epoch: 000 Step/Batch: 000000524 Step mean time: 0004ms 
Loss: 2.9061646 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 20:52:54 2018: Epoch: 000 Step/Batch: 000000525 Step mean time: 0008ms 
Loss: 3.0212326 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 20:52:59 2018: Elapsed time: 0011ms --- Loss: 3.0335603 Validation accuracy: 0.1000
INFO:tensorflow:Sat Nov 17 20:53:02 2018: Epoch: 000 Step/Batch: 000000526 Step mean time: 0013ms 
Loss: 3.0342224 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 20:53:06 2018: Epoch: 000 Step/Batch: 000000527 Step mean time: 0017ms 
Loss: 3.0235653 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 20:53:09 2018: Epoch: 000 Step/Batch: 000000528 Step mean time: 0021ms 
Loss: 3.0299811 Training accuracy: 0.1250
INFO:tensorflow:Sat Nov 17 20:53:13 2018: Epoch: 000 Step/Batch: 000000529 Step mean time: 0025ms 
Loss: 3.0080168 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 20:53:16 2018: E

INFO:tensorflow:Sat Nov 17 21:00:07 2018: Epoch: 003 Step/Batch: 000000572 Step mean time: 0369ms 
Loss: 2.5376608 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:00:17 2018: Epoch: 003 Step/Batch: 000000573 Step mean time: 0379ms 
Loss: 2.5701072 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:00:26 2018: Epoch: 003 Step/Batch: 000000574 Step mean time: 0388ms 
Loss: 2.5485659 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 21:00:31 2018: Epoch: 003 Step/Batch: 000000575 Step mean time: 0392ms 
Loss: 2.3363218 Training accuracy: 0.2500
INFO:tensorflow:Sat Nov 17 21:00:42 2018: Elapsed time: 0474ms --- Loss: 2.6659317 Validation accuracy: 0.1600
INFO:tensorflow:Sat Nov 17 21:00:51 2018: Epoch: 004 Step/Batch: 000000576 Step mean time: 0401ms 
Loss: 2.4546812 Training accuracy: 0.3125
INFO:tensorflow:Sat Nov 17 21:01:01 2018: Epoch: 004 Step/Batch: 000000577 Step mean time: 0410ms 
Loss: 2.7988813 Training accuracy: 0.0625
INFO:tensorflow:Sat Nov 17 21:01:11 2018: E

INFO:tensorflow:Sat Nov 17 21:09:37 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:09:47 2018: Epoch: 007 Step/Batch: 000000621 Step mean time: 0795ms 
Loss: 2.2977057 Training accuracy: 0.5625
INFO:tensorflow:Sat Nov 17 21:09:57 2018: Epoch: 007 Step/Batch: 000000622 Step mean time: 0804ms 
Loss: 2.0453110 Training accuracy: 0.6875
INFO:tensorflow:Sat Nov 17 21:10:08 2018: Epoch: 007 Step/Batch: 000000623 Step mean time: 0813ms 
Loss: 2.4379005 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:10:18 2018: Epoch: 007 Step/Batch: 000000624 Step mean time: 0821ms 
Loss: 2.1597006 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:10:28 2018: Epoch: 007 Step/Batch: 000000625 Step mean time: 0830ms 
Loss: 2.3942926 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 21:10:41 2018: Elapsed time: 1073ms --- Loss: 2.4570012 Validation accuracy: 0.2800
INFO:tensorflow:Sat Nov 17 21:10:51 2018: Epoch: 007 Step/Batch: 000000626 Step mean time: 0838ms 
Loss: 2.1038661 Training ac

INFO:tensorflow:Sat Nov 17 21:19:46 2018: Epoch: 011 Step/Batch: 000000670 Step mean time: 1176ms 
Loss: 1.8499116 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 21:19:59 2018: Elapsed time: 1631ms --- Loss: 2.6637340 Validation accuracy: 0.2000
INFO:tensorflow:Sat Nov 17 21:20:00 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:20:10 2018: Epoch: 011 Step/Batch: 000000671 Step mean time: 1184ms 
Loss: 1.8731762 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 21:20:21 2018: Epoch: 011 Step/Batch: 000000672 Step mean time: 1192ms 
Loss: 1.6822212 Training accuracy: 0.5000
INFO:tensorflow:Sat Nov 17 21:20:32 2018: Epoch: 011 Step/Batch: 000000673 Step mean time: 1200ms 
Loss: 1.7546399 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 21:20:42 2018: Epoch: 011 Step/Batch: 000000674 Step mean time: 1207ms 
Loss: 1.6579763 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 21:20:54 2018: Epoch: 011 Step/Batch: 000000675 Step mean time: 1216ms 
Loss: 2.0095317 Training ac

INFO:tensorflow:Sat Nov 17 21:30:10 2018: Epoch: 015 Step/Batch: 000000719 Step mean time: 1515ms 
Loss: 1.0148653 Training accuracy: 0.8125
INFO:tensorflow:Sat Nov 17 21:30:20 2018: Epoch: 015 Step/Batch: 000000720 Step mean time: 1522ms 
Loss: 1.6286139 Training accuracy: 0.5625
INFO:tensorflow:Sat Nov 17 21:30:33 2018: Elapsed time: 2265ms --- Loss: 3.1113844 Validation accuracy: 0.2000
INFO:tensorflow:Sat Nov 17 21:30:33 2018: Variables saved
INFO:tensorflow:Sat Nov 17 21:30:43 2018: Epoch: 015 Step/Batch: 000000721 Step mean time: 1528ms 
Loss: 1.3900125 Training accuracy: 0.6250
INFO:tensorflow:Sat Nov 17 21:30:53 2018: Epoch: 015 Step/Batch: 000000722 Step mean time: 1535ms 
Loss: 1.3904812 Training accuracy: 0.8125
INFO:tensorflow:Sat Nov 17 21:31:03 2018: Epoch: 015 Step/Batch: 000000723 Step mean time: 1541ms 
Loss: 1.6206560 Training accuracy: 0.4375
INFO:tensorflow:Sat Nov 17 21:31:13 2018: Epoch: 015 Step/Batch: 000000724 Step mean time: 1547ms 
Loss: 1.3186312 Training ac

In [23]:
newnet = AlexNet(training = True)


Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but note that `track_layer` is no longer necessary or supported. Instead, `Layer` instances are tracked on attribute assignment (see the section of `tf.keras.Model`'s documentation on subclassing). Since the output of `track_layer` is often assigned to an attribute anyway, most code can be ported by simply removing the `track_layer` calls.

`tf.keras.Model` works with all TensorFlow `Layer` instances, including those from `tf.layers`, but switching to the `tf.keras.layers` versions along with the migration to `tf.keras.Model` is recommended, since it will preserve variable names. Feel free to import it with an alias to avoid excess typing :).


In [24]:
train(newnet)

INFO:tensorflow:Sat Nov 17 22:31:45 2018: Epoch: 000 Step/Batch: 000000012 Step mean time: 0285ms 
Loss: 2.8949721 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:31:50 2018: Epoch: 001 Step/Batch: 000000013 Step mean time: 0495ms 
Loss: 2.7392097 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:31:55 2018: Epoch: 002 Step/Batch: 000000014 Step mean time: 0675ms 
Loss: 2.5607433 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:32:01 2018: Epoch: 003 Step/Batch: 000000015 Step mean time: 0858ms 
Loss: 2.3864205 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:32:06 2018: Epoch: 004 Step/Batch: 000000016 Step mean time: 1002ms 
Loss: 2.2808108 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:32:12 2018: Epoch: 005 Step/Batch: 000000017 Step mean time: 1148ms 
Loss: 2.1603651 Training accuracy: 0.3750
INFO:tensorflow:Sat Nov 17 22:32:17 2018: Epoch: 006 Step/Batch: 000000018 Step mean time: 1272ms 
Loss: 1.9806359 Training accuracy: 0.5000
INFO:tensorfl

In [ ]:
for (_, (test_images, test_labels)) in enumerate(test_dset):
    l = loss(alexnet, 'val', test_images, test_labels)
    a = accuracy(alexnet, 'val', test_images, test_labels).numpy()
log_msg('Final Test Loss: {:.7f} Test accuracy: {:.4f}'.format(l, a))

In [29]:
import cv2

In [81]:
cifar10 = tf.keras.datasets.cifar10.load_data()
(X_train, y_train), _ = cifar10
X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.int32).flatten()
    
mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
X_train_normalised = (X_train - mean_pixel) / std_pixel

In [82]:
import random
train_examples_with_label = {}
for i in range(10):
    train_examples_with_label[i] = np.array([index for index, value in enumerate(y_train) if value == i])
train_labels = []
for i in range(10):
    train_labels = train_labels + random.sample(list(train_examples_with_label[i]), int(1))
random.shuffle(train_labels)
X_train = X_train[train_labels]
y_train = y_train[train_labels]
X_train_normalised = X_train_normalised[train_labels]

In [83]:
image_path = './images'
if not os.path.exists(image_path):
    os.makedirs(image_path)
def resize_datasets(X):
    X_new = []
    for i in range(X.shape[0]):
        img = scipy.ndimage.interpolation.zoom(X[i], (8.0, 8.0, 1.0))
        X_new.append(img)
    X_new = np.asarray(X_new, dtype=np.float32)
    return X_new.reshape([X.shape[0], 256, 256, 3])
X_train_resized = resize_datasets(X_train)
X_train_normalised_resized = resize_datasets(X_train_normalised)

In [84]:
train_dset = Dataset(X_train_normalised_resized, y_train, batch_size=10, shuffle=False)

In [ ]:
newnet = AlexNet(training = True)

In [260]:
train(newnet)

INFO:tensorflow:Sat Nov 17 23:46:09 2018: Epoch: 000 Step/Batch: 000000132 Step mean time: 0011ms 
Loss: 1.5615523 Training accuracy: 0.7000
INFO:tensorflow:Sat Nov 17 23:46:11 2018: Epoch: 001 Step/Batch: 000000133 Step mean time: 0022ms 
Loss: 0.5384739 Training accuracy: 1.0000
INFO:tensorflow:Sat Nov 17 23:46:13 2018: Epoch: 002 Step/Batch: 000000134 Step mean time: 0032ms 
Loss: 1.5018649 Training accuracy: 0.8000
INFO:tensorflow:Sat Nov 17 23:46:16 2018: Epoch: 003 Step/Batch: 000000135 Step mean time: 0042ms 
Loss: 0.9185445 Training accuracy: 0.9000
INFO:tensorflow:Sat Nov 17 23:46:18 2018: Epoch: 004 Step/Batch: 000000136 Step mean time: 0053ms 
Loss: 0.6333767 Training accuracy: 0.9000
INFO:tensorflow:Sat Nov 17 23:46:20 2018: Epoch: 005 Step/Batch: 000000137 Step mean time: 0062ms 
Loss: 0.5300307 Training accuracy: 1.0000
INFO:tensorflow:Sat Nov 17 23:46:22 2018: Epoch: 006 Step/Batch: 000000138 Step mean time: 0072ms 
Loss: 0.5266472 Training accuracy: 1.0000
INFO:tensorfl

In [261]:
image_path = './images'
if not os.path.exists(image_path):
    os.makedirs(image_path)
for i in range(X_train_resized.shape[0]):
    image_path_local = image_path + '/' + str(i)
    if not os.path.exists(image_path_local):
        os.makedirs(image_path_local)
    curr_input_image = X_train_resized[i, :, :, :]
    curr_input_image = tf.reshape(curr_input_image/255.0, (256, 256, 3))
    cv2.imwrite(image_path_local + '/input_image_' + str(i) + '.png', curr_input_image.numpy()*255.0)
    _,attention1, attention2 = newnet(tf.reshape(X_train_normalised_resized[i, :, :, :], (1, 256, 256, 3)))
    attention1 = np.array(np.repeat(attention1.numpy().reshape(15,15,1), 3, axis = 2))
    attention2 = np.array(np.repeat(attention2.numpy().reshape(15,15,1), 3, axis = 2))
    attention1 = cv2.normalize(attention1, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    attention2 = cv2.normalize(attention2, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    cv2.imwrite(image_path_local + '/attention1_' + str(i) + '.png', attention1)
    cv2.imwrite(image_path_local + '/attention2_' + str(i) + '.png', attention2)
    heatmap1 = cv2.resize(cv2.imread(image_path_local + '/attention1_' + str(i) + '.png'), (256,256))
    heatmap2 = cv2.resize(cv2.imread(image_path_local + '/attention2_' + str(i) + '.png'), (256,256))
    image = cv2.resize(cv2.imread(image_path_local + '/input_image_' + str(i) + '.png'), (256,256))
    heatmap1 = cv2.applyColorMap(heatmap1, cv2.COLORMAP_JET)
    heatmap2 = cv2.applyColorMap(heatmap2, cv2.COLORMAP_JET)
    overlay1 = cv2.addWeighted(heatmap1, 0.6, image, 0.4, 0)
    overlay2 = cv2.addWeighted(heatmap2, 0.6, image, 0.4, 0)
    cv2.imwrite(image_path_local + '/overlay_attention1_' + str(i) + '.png', overlay1)
    cv2.imwrite(image_path_local + '/overlay_attention2_' + str(i) + '.png', overlay2)